In [1]:
### 필요한 모듈 IMPORT
import pandas as pd
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [2]:
################################################
#####BASE MODEL 확인을 위한 간단한 전처리#########
################################################

# 파일 읽어오기
a=pd.read_csv(r"C:\Users\moonp\ASAC과정\ASAC_ml_project\web_crawling\merge_dataset\raw_merge.csv",sep="\t")
### METACRITIC 점수 표기 --> 메타크리틱 점수가 없는 경우 -1로 대체
a.metacritic=a.metacritic.fillna(-1)
tmp=a.dropna(axis=0)
### 동접자수 전처리 과정 동접자수 없는 경우 0으로 표기
tmp["24_Hour_Peak"].replace("Invalid","0",inplace=True)
tmp["All_time_peak"].replace("Invalid","0",inplace=True)
tmp["24_Hour_Peak"]=tmp["24_Hour_Peak"].astype(int)
tmp["All_time_peak"]=tmp["All_time_peak"].astype(int)

### is free 전처리 기존의 FALSE,TRUE --> 0,1로 바꾸어서
tmp.is_free.replace("False","0",inplace=True)
tmp.is_free.replace("0.0","0",inplace=True)
tmp.is_free.replace("True","1",inplace=True)
tmp.is_free.replace("1.0","1",inplace=True)
tmp.is_free=tmp.is_free.astype(int)
tmp.is_free=tmp.is_free.astype(int)


### TARGET CLASS를 4개로 재분류
No_interest_lst=['No user reviews','8 user reviews','7 user reviews','4 user reviews','9 user reviews','5 user reviews','6 user reviews','2 user reviews','3 user reviews','1 user reviews','None']
Positive_lst=['Overwhelmingly Positive','Very Positive','Positive','Mostly Positive']
Negative_lst=['Negative','Very Negative','Overwhelmingly Negative','Mostly Negative']
tmp.Review.replace(No_interest_lst,"No_interest",inplace=True)
tmp.Review.replace(Positive_lst,"Positive",inplace=True)
tmp.Review.replace(Negative_lst,"Negative",inplace=True)

### TARGET LABEL ENCODING
le=preprocessing.LabelEncoder()
le.fit(np.array(tmp.Review))
y_true=le.fit_transform(tmp.Review)
print(le.classes_)

['Mixed' 'Negative' 'No_interest' 'Positive']


In [3]:
a.columns

Index(['appid', '24_Hour_Peak', 'All_time_peak', 'positive', 'negative',
       'average_forever', 'average_2weeks', 'median_forever', 'median_2weeks',
       'price', 'initialprice', 'genre', 'num_lang', 'required_age', 'is_free',
       'controller_support', 'dlc', 'metacritic', 'metacritic_tf', 'windows',
       'mac', 'linux', 'platforms_num', 'recommendations', 'achievements',
       'new_date', 'days_after_release', 'Review'],
      dtype='object')

In [4]:
##########GENRE 없이 BASE MODEL 학습 RF로만 간단하게 구성 #########

##########데이터 전처리 #######################################
x_no_genre=tmp.drop(columns=["appid","positive","negative","genre","new_date","Review","recommendations"])
x_train, x_val, y_train, y_val = train_test_split(
    x_no_genre,    # 문제 --> train/val
    y_true,    # 정답지
    test_size= 0.2, # train 데이터 중에서 20프로만 중간 검사용으로 빼두자..
    random_state= 1234, # 옵션 : 나중에 재현성을 위해서 seed값.
    stratify = y_true # 옵션 : 나눌 때 비율을 유지할지 말지....
)


In [ ]:
kfold = KFold( n_splits=5, random_state=1234, shuffle=True)
# cv = 5 ---> 내가 run할 때 마다 셋들이 5개이기는 하지만 구성이 제각가..
# cv = kold ---> 언제, 어디서 하더라도 5개 셋이, 다 고정으로 동일하게 적용!!!
# 1) 모델
rf = RandomForestClassifier(n_jobs=-1, random_state=1234,verbose=2)
# 2) 파라미터의 조합 RGS --> 여러개가 있지만,,대표적인 것만 세팅..
parameters = {
    # 몇 개의 모델을 활용할지 
    "n_estimators" : [10, 100, 500, 1000], # 7
    # 세로에 대해서 어떻게 샘플링을 할지.
    "max_features" : ["sqrt","log2", None], # 3
    "min_samples_split" : [2, 5, 10],
    # 각 모델이 dt의 overfit을 조절하기
    "max_depth" : [2,5,10,20,100], # 10
    # min sample leaf
    "min_samples_leaf" : [1, 2, 4],
    "criterion" : ["gini", "entropy", "log_loss"],
    # 기타 여러 parameters 세팅 가능함....
}
# 3) RGS --> 몇 번 반복할지 
n_iter = 100
# 4) RGSCV 세팅.
rf_kfold_rgs = RandomizedSearchCV( rf, 
                                  param_distributions= parameters,
                                  cv = kfold,
                                  scoring = "accuracy",
                                  n_jobs= -1,
                                  random_state=1234,
                                  n_iter = n_iter,verbose=3
                                  )
# 5) 학습
rf_kfold_rgs.fit(x_train, y_train)

In [ ]:
rf_kfold_rgs.best_score_
#

In [ ]:
rf_kfold_rgs.best_estimator_

In [9]:
##########Louvain Network Clustering (4Class)#########
Class1_lst=['Action','Adventure','Casual','EarlyAccess','Indie','RPG','Simulation','Strategy']
Class2_lst=['ActionGames', 'FreetoPlay', 'MassivelyMultiplayer', 'Racing', 'Sports']
Class3_lst=['Gore', 'Nudity', 'SexualContent', 'Violent']
Class4_lst=['360V','Accounting','Animation&Modeling','AudioProduction','Design&Illustration','Documentary',
'Education','Episodic','GameDevelopment','Movie','PhotoEditing','Short','SoftwareTraining','Tutorial',
'Utilities','VideoProduction','WebPublishing']

###########Genre Classification##################################
df_genre=pd.DataFrame(tmp.genre)
genre_pre=[]
for i in range(len(df_genre)):
    a=df_genre.iloc[i,0].replace(" ","").split(",")
    category=[0 for i in range(4)]
    for j in range(len(a)):
        if a[j] in Class1_lst:
            category[0]+=1
        elif a[j] in Class2_lst:
            category[1]+=1
        elif a[j] in Class3_lst:
            category[2]+=1
        elif a[j] in Class4_lst:
            category[3]+=1
    genre_pre.append(category)
genre_pre=pd.DataFrame(genre_pre)
genre_pre.columns=["Genre_class1","Genre_class2","Genre_class3","Genre_class4"]

x_no_genre=tmp.drop(columns=["appid","positive","negative","genre","new_date","Review"])
x_no_genre.reset_index(inplace=True)

louvain_x=pd.concat([x_no_genre,genre_pre],axis=1)
louvain_x=louvain_x.drop(columns=["index"])
x_train, x_val, y_train, y_val = train_test_split(
    louvain_x,    # 문제 --> train/val
    y_true,    # 정답지
    test_size= 0.2, # train 데이터 중에서 20프로만 중간 검사용으로 빼두자..
    random_state= 1234, # 옵션 : 나중에 재현성을 위해서 seed값.
    stratify = y_true # 옵션 : 나눌 때 비율을 유지할지 말지....
)
kfold = KFold( n_splits=5, random_state=1234, shuffle=True)
# cv = 5 ---> 내가 run할 때 마다 셋들이 5개이기는 하지만 구성이 제각가..
# cv = kold ---> 언제, 어디서 하더라도 5개 셋이, 다 고정으로 동일하게 적용!!!
# 1) 모델
rf = RandomForestClassifier(n_jobs=-1, random_state=1234,verbose=2)
# 2) 파라미터의 조합 RGS --> 여러개가 있지만,,대표적인 것만 세팅..
parameters = {
    # 몇 개의 모델을 활용할지 
    "n_estimators" : [10, 100, 500, 1000], # 7
    # 세로에 대해서 어떻게 샘플링을 할지.
    "max_features" : ["sqrt","log2", None], # 3
    "min_samples_split" : [2, 5, 10],
    # 각 모델이 dt의 overfit을 조절하기
    "max_depth" : [2,5,10,20,100], # 10
    # min sample leaf
    "min_samples_leaf" : [1, 2, 4],
    "criterion" : ["gini", "entropy", "log_loss"],
    # 기타 여러 parameters 세팅 가능함....
}
# 3) RGS --> 몇 번 반복할지 
n_iter = 100
# 4) RGSCV 세팅.
rf_louvain = RandomizedSearchCV( rf, 
                                  param_distributions= parameters,
                                  cv = kfold,
                                  scoring = "accuracy",
                                  n_jobs= -1,
                                  random_state=1234,
                                  n_iter = n_iter,verbose=3
                                  )
# 5) 학습
rf_louvain.fit(x_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   46.2s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed: 12.2min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 21.8min finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000building tree 31 of 1000

building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.3s


building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building tree 41 of 1000building tree 42 of 1000

building tree 43 of 1000
building tree 44 of 1000
building tree 45 of 1000
building tree 46 of 1000
building tree 47 of 1000
building tree 48 of 1000
building tree 49 of 1000
building tree 50 of 1000
building tree 51 of 1000building tree 52 of 1000

building tree 53 of 1000
building tree 54 of 1000
building tree 55 of 1000
building tree 56 of 1000
building tree 57 of 1000
building tree 58 of 1000
building tree 59 of 1000building tree 60 of 1000building tree 61 of 1000


building tree 62 of 1000
building tree 63 of 1000
building tree 64 of 1000
building tree 65 of 1000
building tree 66 of 1000
building tree 67 of 1000
building tree 68 of 1000
building tree 69 of 1000
building tree 70 of 1000
building tree 71 of 1000
building tree 72 of 1000
building tree 73 of 1000
building tree 74 of 1000
building tree 75 of 1000


...


building tree 366 of 1000
building tree 367 of 1000
building tree 368 of 1000
building tree 369 of 1000
building tree 370 of 1000
building tree 371 of 1000
building tree 372 of 1000
building tree 373 of 1000
building tree 374 of 1000
building tree 375 of 1000
building tree 376 of 1000
building tree 377 of 1000
building tree 378 of 1000
building tree 379 of 1000
building tree 380 of 1000
building tree 381 of 1000building tree 382 of 1000

building tree 383 of 1000
building tree 384 of 1000
building tree 385 of 1000
building tree 386 of 1000
building tree 387 of 1000
building tree 388 of 1000
building tree 389 of 1000
building tree 390 of 1000
building tree 391 of 1000
building tree 392 of 1000
building tree 393 of 1000
building tree 394 of 1000
building tree 395 of 1000
building tree 396 of 1000
building tree 397 of 1000
building tree 398 of 1000building tree 399 of 1000

building tree 400 of 1000
building tree 401 of 1000
building tree 402 of 1000
building tree 403 of 1000
building tr

[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:    7.7s


building tree 656 of 1000building tree 657 of 1000

building tree 658 of 1000
building tree 659 of 1000
building tree 660 of 1000
building tree 661 of 1000
building tree 662 of 1000
building tree 663 of 1000
building tree 664 of 1000
building tree 665 of 1000
building tree 666 of 1000
building tree 667 of 1000
building tree 668 of 1000
building tree 669 of 1000
building tree 670 of 1000
building tree 671 of 1000
building tree 672 of 1000
building tree 673 of 1000building tree 674 of 1000

building tree 675 of 1000
building tree 676 of 1000
building tree 677 of 1000
building tree 678 of 1000
building tree 679 of 1000
building tree 680 of 1000
building tree 681 of 1000
building tree 682 of 1000building tree 683 of 1000

building tree 684 of 1000
building tree 685 of 1000
building tree 686 of 1000
building tree 687 of 1000
building tree 688 of 1000
building tree 689 of 1000
building tree 690 of 1000
building tree 691 of 1000
building tree 692 of 1000
building tree 693 of 1000
building tre

[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   11.8s finished


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=1234, shuffle=True),
                   estimator=RandomForestClassifier(n_jobs=-1,
                                                    random_state=1234,
                                                    verbose=2),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [2, 5, 10, 20, 100],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [10, 100, 500, 1000]},
                   random_state=1234, scoring='accuracy', verbose=3)

In [10]:
rf_louvain.best_score_
#0.6527659994814433

0.6527659994814433

In [11]:
rf_louvain.best_estimator_
# RandomForestClassifier(max_depth=20, max_features='sqrt', min_samples_leaf=4,
#                        n_estimators=1000, n_jobs=-1, random_state=1234,
#                        verbose=2)

RandomForestClassifier(max_depth=20, max_features='sqrt', min_samples_leaf=4,
                       n_estimators=1000, n_jobs=-1, random_state=1234,
                       verbose=2)

In [48]:
##########Louvain Network Clustering (4Class)#########
Class1_lst=['Action','Adventure','Casual','EarlyAccess','Indie','RPG','Simulation','Strategy']
Class2_lst=['ActionGames', 'FreetoPlay', 'MassivelyMultiplayer', 'Racing', 'Sports']
Class3_lst=['Gore', 'Nudity', 'SexualContent', 'Violent']
Class4_lst=['360V','Accounting','Animation&Modeling','AudioProduction','Design&Illustration','Documentary',
'Education','Episodic','GameDevelopment','Movie','PhotoEditing','Short','SoftwareTraining','Tutorial',
'Utilities','VideoProduction','WebPublishing']

###########Genre Classification##################################
df_genre=pd.DataFrame(tmp.genre)
genre_pre=[]
for i in range(len(df_genre)):
    a=df_genre.iloc[i,0].replace(" ","").split(",")
    category=[0 for i in range(4)]
    for j in range(len(a)):
        if a[j] in Class1_lst:
            category[0]+=1
        elif a[j] in Class2_lst:
            category[1]+=1
        elif a[j] in Class3_lst:
            category[2]+=1
        elif a[j] in Class4_lst:
            category[3]+=1
    genre_pre.append(category)
genre_pre=pd.DataFrame(genre_pre)
genre_pre.columns=["Genre_class1","Genre_class2","Genre_class3","Genre_class4"]

x_no_genre=tmp.drop(columns=["appid","positive","negative","genre","new_date","Review"])
x_no_genre.reset_index(inplace=True)

louvain_x=pd.concat([x_no_genre,genre_pre],axis=1)
louvain_x=louvain_x.drop(columns=["index"])
x_train, x_val, y_train, y_val = train_test_split(
    louvain_x,    # 문제 --> train/val
    y_true,    # 정답지
    test_size= 0.2, # train 데이터 중에서 20프로만 중간 검사용으로 빼두자..
    random_state= 1234, # 옵션 : 나중에 재현성을 위해서 seed값.
    stratify = y_true # 옵션 : 나눌 때 비율을 유지할지 말지....
)

rf_parameter=[]
a_tmp=list(x_train.columns)
b_tmp=list(rf_louvain.best_estimator_.feature_importances_)
for i in range(len(a_tmp)):
    rf_parameter.append([a_tmp[i],b_tmp[i]])
rf_parameter=pd.DataFrame(rf_parameter)
rf_parameter=rf_parameter.sort_values(by=[1],ascending=False)

In [49]:
rf_parameter

,0,1
1,All_time_peak,0.183364
19,recommendations,0.111469
21,days_after_release,0.095763
20,achievements,0.095105
6,price,0.073580
7,initialprice,0.070356
8,num_lang,0.066975
0,24_Hour_Peak,0.059618
2,average_forever,0.042610
4,median_forever,0.041544


In [13]:
###########Genre Onehot##################################
df_genre=pd.DataFrame(tmp.genre)
genre_pre=[]
for i in range(len(df_genre)):
    a=df_genre.iloc[i,0].replace(" ","").split(",")
    category=[0 for i in range(4)]
    for j in range(len(a)):
        if a[j] in Class1_lst:
            category[0]+=1
        elif a[j] in Class2_lst:
            category[1]+=1
        elif a[j] in Class3_lst:
            category[2]+=1
        elif a[j] in Class4_lst:
            category[3]+=1
    genre_pre.append(category)
genre_pre=pd.DataFrame(genre_pre)
genre_pre.columns=["Genre_class1","Genre_class2","Genre_class3","Genre_class4"]

In [25]:
##########Genre Category#########
genre_total_lst=dict()
for i in range(len(df_genre)):
    a=df_genre.iloc[i,0].replace(" ","").split(",")
    for j in range(len(a)):
        genre_total_lst[a[j]]=0
genre_total_lst=list(genre_total_lst.keys())

###########Genre Onehot##################################
df_genre=pd.DataFrame(tmp.genre)
genre_pre=[]
for i in range(len(df_genre)):
    a=df_genre.iloc[i,0].replace(" ","").split(",")
    category=[0 for j in range(len(genre_total_lst))]
    for j in range(len(a)):
        for k in range(len(genre_total_lst)):
            if a[j]==genre_total_lst[k]:
                category[k]+=1
    genre_pre.append(category)
genre_pre=pd.DataFrame(genre_pre)
genre_pre.columns=genre_total_lst

In [29]:
##########Genre Category#########
genre_total_lst=dict()
for i in range(len(df_genre)):
    a=df_genre.iloc[i,0].replace(" ","").split(",")
    for j in range(len(a)):
        genre_total_lst[a[j]]=0
genre_total_lst=list(genre_total_lst.keys())

###########Genre Onehot##################################
df_genre=pd.DataFrame(tmp.genre)
genre_pre=[]
for i in range(len(df_genre)):
    a=df_genre.iloc[i,0].replace(" ","").split(",")
    category=[0 for j in range(len(genre_total_lst))]
    for j in range(len(a)):
        for k in range(len(genre_total_lst)):
            if a[j]==genre_total_lst[k]:
                category[k]+=1
    genre_pre.append(category)
genre_pre=pd.DataFrame(genre_pre)
genre_pre.columns=genre_total_lst

x_no_genre=tmp.drop(columns=["appid","positive","negative","genre","new_date","Review"])
x_no_genre.reset_index(inplace=True)

onehot_x=pd.concat([x_no_genre,genre_pre],axis=1)
onehot_x=onehot_x.drop(columns=["index"])
x_train, x_val, y_train, y_val = train_test_split(
    onehot_x,    # 문제 --> train/val
    y_true,    # 정답지
    test_size= 0.2, # train 데이터 중에서 20프로만 중간 검사용으로 빼두자..
    random_state= 1234, # 옵션 : 나중에 재현성을 위해서 seed값.
    stratify = y_true # 옵션 : 나눌 때 비율을 유지할지 말지....
)
kfold = KFold( n_splits=5, random_state=1234, shuffle=True)
# cv = 5 ---> 내가 run할 때 마다 셋들이 5개이기는 하지만 구성이 제각가..
# cv = kold ---> 언제, 어디서 하더라도 5개 셋이, 다 고정으로 동일하게 적용!!!
# 1) 모델
rf = RandomForestClassifier(n_jobs=-1, random_state=1234,verbose=2)
# 2) 파라미터의 조합 RGS --> 여러개가 있지만,,대표적인 것만 세팅..
parameters = {
    # 몇 개의 모델을 활용할지 
    "n_estimators" : [10, 100, 500, 1000], # 7
    # 세로에 대해서 어떻게 샘플링을 할지.
    "max_features" : ["sqrt","log2", None], # 3
    "min_samples_split" : [2, 5, 10],
    # 각 모델이 dt의 overfit을 조절하기
    "max_depth" : [2,5,10,20,100], # 10
    # min sample leaf
    "min_samples_leaf" : [1, 2, 4],
    "criterion" : ["gini", "entropy", "log_loss"],
    # 기타 여러 parameters 세팅 가능함....
}
# 3) RGS --> 몇 번 반복할지 
n_iter = 100
# 4) RGSCV 세팅.
rf_onehot = RandomizedSearchCV( rf, 
                                  param_distributions= parameters,
                                  cv = kfold,
                                  scoring = "accuracy",
                                  n_jobs= -1,
                                  random_state=1234,
                                  n_iter = n_iter,verbose=3
                                  )
# 5) 학습
rf_onehot.fit(x_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   58.8s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed: 19.3min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 32.3min finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 1000building tree 2 of 1000building tree 3 of 1000
building tree 4 of 1000

building tree 5 of 1000

building tree 6 of 1000building tree 7 of 1000
building tree 8 of 1000

building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000building tree 13 of 1000

building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000building tree 23 of 1000

building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.3s



building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000building tree 39 of 1000

building tree 40 of 1000
building tree 41 of 1000
building tree 42 of 1000
building tree 43 of 1000
building tree 44 of 1000
building tree 45 of 1000building tree 46 of 1000

building tree 47 of 1000
building tree 48 of 1000
building tree 49 of 1000
building tree 50 of 1000
building tree 51 of 1000
building tree 52 of 1000
building tree 53 of 1000
building tree 54 of 1000
building tree 55 of 1000
building tree 56 of 1000
building tree 57 of 1000
building tree 58 of 1000
building tree 59 of 1000
building tree 60 of 1000
building tree 61 of 1000
building tree 62 of 1000
building tree 63 of 1000
building tree 64 of 1000building tree 65 of 1000

building tree 66 of 1000
building tree 67 of 1000
building tree 68 of 1000
building tree 69 of 1000
building tree 70 of 1000
building tree 71 of 1000
building tree 72 of 1000
building tree 73 of 1000

...

building tree 370 of 1000
building tree 371 of 1000
building tree 372 of 1000
building tree 373 of 1000
building tree 374 of 1000
building tree 375 of 1000
building tree 376 of 1000
building tree 377 of 1000
building tree 378 of 1000
building tree 379 of 1000
building tree 380 of 1000
building tree 381 of 1000
building tree 382 of 1000
building tree 383 of 1000
building tree 384 of 1000
building tree 385 of 1000
building tree 386 of 1000
building tree 387 of 1000
building tree 388 of 1000building tree 389 of 1000

building tree 390 of 1000
building tree 391 of 1000building tree 392 of 1000

building tree 393 of 1000
building tree 394 of 1000
building tree 395 of 1000building tree 396 of 1000

building tree 397 of 1000
building tree 398 of 1000
building tree 399 of 1000
building tree 400 of 1000
building tree 401 of 1000building tree 402 of 1000

building tree 403 of 1000
building tree 404 of 1000
building tree 405 of 1000
building tree 406 of 1000
building tree 407 of 1000
building tre

[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:    8.8s


building tree 644 of 1000
building tree 645 of 1000
building tree 646 of 1000
building tree 647 of 1000
building tree 648 of 1000
building tree 649 of 1000
building tree 650 of 1000
building tree 651 of 1000
building tree 652 of 1000
building tree 653 of 1000
building tree 654 of 1000
building tree 655 of 1000
building tree 656 of 1000
building tree 657 of 1000
building tree 658 of 1000
building tree 659 of 1000
building tree 660 of 1000
building tree 661 of 1000
building tree 662 of 1000
building tree 663 of 1000
building tree 664 of 1000
building tree 665 of 1000
building tree 666 of 1000
building tree 667 of 1000
building tree 668 of 1000
building tree 669 of 1000
building tree 670 of 1000
building tree 671 of 1000
building tree 672 of 1000
building tree 673 of 1000
building tree 674 of 1000
building tree 675 of 1000
building tree 676 of 1000
building tree 677 of 1000
building tree 678 of 1000building tree 679 of 1000

building tree 680 of 1000building tree 681 of 1000

building tre

[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   13.8s finished


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=1234, shuffle=True),
                   estimator=RandomForestClassifier(n_jobs=-1,
                                                    random_state=1234,
                                                    verbose=2),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [2, 5, 10, 20, 100],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [10, 100, 500, 1000]},
                   random_state=1234, scoring='accuracy', verbose=3)

In [30]:
rf_onehot.best_score_
#0.6583875444151083

0.6583875444151083

In [31]:
rf_onehot.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=100, max_features='log2',
                       min_samples_leaf=2, min_samples_split=10,
                       n_estimators=1000, n_jobs=-1, random_state=1234,
                       verbose=2)

In [32]:
x_train

,24_Hour_Peak,All_time_peak,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,num_lang,required_age,...,AudioProduction,PhotoEditing,GameDevelopment,ActionGames,Movie,Documentary,Episodic,Short,Tutorial,360V
21280,0,0,0,0,0,0,0.00,0.00,1,0.0,...,0,0,0,0,0,0,0,0,0,0
23372,0,0,0,0,0,0,4.99,4.99,1,0.0,...,0,0,0,0,0,0,0,0,0,0
14647,0,0,0,0,0,0,1.99,1.99,1,0.0,...,0,0,0,0,0,0,0,0,0,0
20419,5,153,0,0,0,0,29.99,29.99,15,0.0,...,0,0,0,0,0,0,0,0,0,0
27558,0,0,0,0,0,0,1.99,1.99,7,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36564,0,0,0,0,0,0,0.99,0.99,1,0.0,...,0,0,0,0,0,0,0,0,0,0
12196,0,0,0,0,0,0,4.99,4.99,4,0.0,...,0,0,0,0,0,0,0,0,0,0
31817,0,0,0,0,0,0,0.00,0.00,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3517,0,0,0,0,0,0,4.99,4.99,1,0.0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
rf_parameter=[]
a_tmp=list(x_train.columns)
b_tmp=list(rf_onehot.best_estimator_.feature_importances_)
for i in range(len(a_tmp)):
    rf_parameter.append([a_tmp[i],b_tmp[i]])
rf_parameter=pd.DataFrame(rf_parameter)
rf_parameter=rf_parameter.sort_values(by=[1],ascending=False)

In [35]:
rf_parameter

,0,1
1,All_time_peak,1.328695e-01
21,days_after_release,9.834561e-02
19,recommendations,8.718850e-02
20,achievements,7.898956e-02
7,initialprice,6.913335e-02
6,price,6.838595e-02
8,num_lang,6.314695e-02
0,24_Hour_Peak,4.814441e-02
4,median_forever,4.180354e-02
2,average_forever,3.951857e-02


In [37]:
tmp

,appid,24_Hour_Peak,All_time_peak,positive,negative,average_forever,average_2weeks,median_forever,median_2weeks,price,...,metacritic_tf,windows,mac,linux,platforms_num,recommendations,achievements,new_date,days_after_release,Review
0,294100,20930,60742,157344,3038,12492,1401,5368,921,34.99,...,1,1.0,1.0,1.0,3.0,134967.0,0.0,2018-10-17,1675,Positive
1,960090,13428,53891,246979,6441,4018,277,1532,62,13.99,...,0,1.0,1.0,0.0,2.0,222889.0,146.0,2018-12-18,1613,Positive
2,504230,1234,4148,71428,1686,1316,310,370,582,19.99,...,1,1.0,1.0,1.0,3.0,66397.0,32.0,2018-01-25,1940,Positive
3,588650,4553,14880,128085,3722,1841,183,868,213,14.99,...,1,1.0,1.0,1.0,3.0,113184.0,121.0,2018-08-06,1747,Positive
4,264710,4205,50876,221972,7800,1943,313,941,277,29.99,...,1,1.0,1.0,0.0,2.0,208334.0,17.0,2018-01-24,1941,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46247,1995280,0,0,0,0,0,0,0,0,14.99,...,0,1.0,0.0,0.0,1.0,0.0,0.0,2022-12-02,168,Negative
46248,2098860,0,0,0,11,0,0,0,0,9.99,...,0,1.0,0.0,0.0,1.0,0.0,0.0,2022-08-09,283,Negative
46249,2154770,814,1176,0,0,0,0,0,0,0.00,...,0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-01,169,Negative
46250,1915410,0,1,0,16,0,0,0,0,2.99,...,0,1.0,0.0,0.0,1.0,0.0,0.0,2022-04-26,388,Negative


In [42]:
pd.DataFrame(y_true).value_counts()

3    19419
2    19096
0     6575
1     1162
dtype: int64

In [54]:
##########Louvain Network Clustering (4Class)#########
Class1_lst=['Action','Adventure','Casual','EarlyAccess','Indie','RPG','Simulation','Strategy']
Class2_lst=['ActionGames', 'FreetoPlay', 'MassivelyMultiplayer', 'Racing', 'Sports']
Class3_lst=['Gore', 'Nudity', 'SexualContent', 'Violent']
Class4_lst=['360V','Accounting','Animation&Modeling','AudioProduction','Design&Illustration','Documentary',
'Education','Episodic','GameDevelopment','Movie','PhotoEditing','Short','SoftwareTraining','Tutorial',
'Utilities','VideoProduction','WebPublishing']

###########Genre Classification##################################
df_genre=pd.DataFrame(tmp.genre)
genre_pre=[]
for i in range(len(df_genre)):
    a=df_genre.iloc[i,0].replace(" ","").split(",")
    category=[0 for i in range(4)]
    for j in range(len(a)):
        if a[j] in Class1_lst:
            category[0]+=1
        elif a[j] in Class2_lst:
            category[1]+=1
        elif a[j] in Class3_lst:
            category[2]+=1
        elif a[j] in Class4_lst:
            category[3]+=1
    genre_pre.append(category)
genre_pre=pd.DataFrame(genre_pre)
genre_pre.columns=["Genre_class1","Genre_class2","Genre_class3","Genre_class4"]

x_no_genre=tmp.drop(columns=["appid","positive","negative","genre","new_date","Review"])
x_no_genre.reset_index(inplace=True)

louvain_x=pd.concat([x_no_genre,genre_pre],axis=1)
louvain_x=louvain_x.drop(columns=["index"])

##########Genre Category#########
genre_total_lst=dict()
for i in range(len(df_genre)):
    a=df_genre.iloc[i,0].replace(" ","").split(",")
    for j in range(len(a)):
        genre_total_lst[a[j]]=0
genre_total_lst=list(genre_total_lst.keys())

###########Genre Onehot##################################
df_genre=pd.DataFrame(tmp.genre)
genre_pre=[]
for i in range(len(df_genre)):
    a=df_genre.iloc[i,0].replace(" ","").split(",")
    category=[0 for j in range(len(genre_total_lst))]
    for j in range(len(a)):
        for k in range(len(genre_total_lst)):
            if a[j]==genre_total_lst[k]:
                category[k]+=1
    genre_pre.append(category)
genre_pre=pd.DataFrame(genre_pre)
genre_pre.columns=genre_total_lst

x_no_genre=tmp.drop(columns=["appid","positive","negative","genre","new_date","Review"])
x_no_genre.reset_index(inplace=True)

onehot_x=pd.concat([louvain_x,genre_pre],axis=1)
# onehot_x=onehot_x.drop(columns=["index"])


In [55]:
onehot_x

,24_Hour_Peak,All_time_peak,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,num_lang,required_age,...,AudioProduction,PhotoEditing,GameDevelopment,ActionGames,Movie,Documentary,Episodic,Short,Tutorial,360V
0,20930,60742,12492,1401,5368,921,34.99,34.99,35,0.0,...,0,0,0,0,0,0,0,0,0,0
1,13428,53891,4018,277,1532,62,13.99,13.99,24,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1234,4148,1316,310,370,582,19.99,19.99,14,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4553,14880,1841,183,868,213,14.99,24.99,21,0.0,...,0,0,0,0,0,0,0,0,0,0
4,4205,50876,1943,313,941,277,29.99,29.99,38,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46247,0,0,0,0,0,0,14.99,14.99,4,0.0,...,0,0,0,0,0,0,0,0,0,0
46248,0,0,0,0,0,0,9.99,9.99,1,0.0,...,0,0,0,0,0,0,0,0,0,0
46249,814,1176,0,0,0,0,0.00,0.00,0,0.0,...,0,0,0,0,0,0,0,0,0,0
46250,0,1,0,0,0,0,2.99,2.99,1,0.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
onehot_x.columns

Index(['24_Hour_Peak', 'All_time_peak', 'average_forever', 'average_2weeks',
       'median_forever', 'median_2weeks', 'price', 'initialprice', 'num_lang',
       'required_age', 'is_free', 'controller_support', 'dlc', 'metacritic',
       'metacritic_tf', 'windows', 'mac', 'linux', 'platforms_num',
       'recommendations', 'achievements', 'days_after_release', 'Indie',
       'Simulation', 'Strategy', 'Action', 'Adventure', 'RPG', 'EarlyAccess',
       'Casual', 'FreetoPlay', 'Sports', 'Racing', 'MassivelyMultiplayer', 'N',
       'Violent', 'Gore', 'SexualContent', 'Animation&Modeling',
       'Design&Illustration', 'Education', 'SoftwareTraining', 'Utilities',
       'WebPublishing', 'VideoProduction', 'Nudity', 'Accounting',
       'AudioProduction', 'PhotoEditing', 'GameDevelopment', 'ActionGames',
       'Movie', 'Documentary', 'Episodic', 'Short', 'Tutorial', '360V'],
      dtype='object')

In [56]:
onehot_x.columns

Index(['24_Hour_Peak', 'All_time_peak', 'average_forever', 'average_2weeks',
       'median_forever', 'median_2weeks', 'price', 'initialprice', 'num_lang',
       'required_age', 'is_free', 'controller_support', 'dlc', 'metacritic',
       'metacritic_tf', 'windows', 'mac', 'linux', 'platforms_num',
       'recommendations', 'achievements', 'days_after_release', 'Genre_class1',
       'Genre_class2', 'Genre_class3', 'Genre_class4', 'Indie', 'Simulation',
       'Strategy', 'Action', 'Adventure', 'RPG', 'EarlyAccess', 'Casual',
       'FreetoPlay', 'Sports', 'Racing', 'MassivelyMultiplayer', 'N',
       'Violent', 'Gore', 'SexualContent', 'Animation&Modeling',
       'Design&Illustration', 'Education', 'SoftwareTraining', 'Utilities',
       'WebPublishing', 'VideoProduction', 'Nudity', 'Accounting',
       'AudioProduction', 'PhotoEditing', 'GameDevelopment', 'ActionGames',
       'Movie', 'Documentary', 'Episodic', 'Short', 'Tutorial', '360V'],
      dtype='object')

In [57]:
x_train, x_val, y_train, y_val = train_test_split(
    onehot_x,    # 문제 --> train/val
    y_true,    # 정답지
    test_size= 0.2, # train 데이터 중에서 20프로만 중간 검사용으로 빼두자..
    random_state= 1234, # 옵션 : 나중에 재현성을 위해서 seed값.
    stratify = y_true # 옵션 : 나눌 때 비율을 유지할지 말지....
)
kfold = KFold( n_splits=5, random_state=1234, shuffle=True)
# cv = 5 ---> 내가 run할 때 마다 셋들이 5개이기는 하지만 구성이 제각가..
# cv = kold ---> 언제, 어디서 하더라도 5개 셋이, 다 고정으로 동일하게 적용!!!
# 1) 모델
rf = RandomForestClassifier(n_jobs=-1, random_state=1234,verbose=2)
# 2) 파라미터의 조합 RGS --> 여러개가 있지만,,대표적인 것만 세팅..
parameters = {
    # 몇 개의 모델을 활용할지 
    "n_estimators" : [10, 100, 500, 1000], # 7
    # 세로에 대해서 어떻게 샘플링을 할지.
    "max_features" : ["sqrt","log2", None], # 3
    "min_samples_split" : [2, 5, 10],
    # 각 모델이 dt의 overfit을 조절하기
    "max_depth" : [2,5,10,20,100], # 10
    # min sample leaf
    "min_samples_leaf" : [1, 2, 4],
    "criterion" : ["gini", "entropy", "log_loss"],
    # 기타 여러 parameters 세팅 가능함....
}
# 3) RGS --> 몇 번 반복할지 
n_iter = 100
# 4) RGSCV 세팅.
rf_onehot_class4 = RandomizedSearchCV( rf, 
                                  param_distributions= parameters,
                                  cv = kfold,
                                  scoring = "accuracy",
                                  n_jobs= -1,
                                  random_state=1234,
                                  n_iter = n_iter,verbose=3
                                  )
# 5) 학습
rf_onehot_class4 .fit(x_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed: 21.1min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 35.1min finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 1000building tree 2 of 1000

building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000building tree 8 of 1000

building tree 9 of 1000
building tree 10 of 1000building tree 11 of 1000

building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000building tree 22 of 1000

building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000building tree 31 of 1000building tree 32 of 1000


building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000building tree 38 of 1000building tree 39 of 1000

building tree 40 of 1000



[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.5s


building tree 41 of 1000
building tree 42 of 1000
building tree 43 of 1000
building tree 44 of 1000
building tree 45 of 1000building tree 46 of 1000

building tree 47 of 1000
building tree 48 of 1000
building tree 49 of 1000
building tree 50 of 1000building tree 51 of 1000building tree 52 of 1000


building tree 53 of 1000building tree 54 of 1000
building tree 55 of 1000

building tree 56 of 1000
building tree 57 of 1000
building tree 58 of 1000
building tree 59 of 1000
building tree 60 of 1000
building tree 61 of 1000
building tree 62 of 1000
building tree 63 of 1000
building tree 64 of 1000
building tree 65 of 1000
building tree 66 of 1000
building tree 67 of 1000
building tree 68 of 1000
building tree 69 of 1000building tree 70 of 1000

building tree 71 of 1000building tree 72 of 1000

building tree 73 of 1000
building tree 74 of 1000building tree 75 of 1000

building tree 76 of 1000
building tree 77 of 1000building tree 78 of 1000

building tree 79 of 1000
building tree 80 of 1000


...

building tree 363 of 1000
building tree 364 of 1000
building tree 365 of 1000
building tree 366 of 1000building tree 367 of 1000

building tree 368 of 1000building tree 369 of 1000
building tree 370 of 1000

building tree 371 of 1000
building tree 372 of 1000
building tree 373 of 1000
building tree 374 of 1000
building tree 375 of 1000
building tree 376 of 1000
building tree 377 of 1000
building tree 378 of 1000
building tree 379 of 1000
building tree 380 of 1000building tree 381 of 1000building tree 382 of 1000

building tree 383 of 1000

building tree 384 of 1000
building tree 385 of 1000
building tree 386 of 1000
building tree 387 of 1000
building tree 388 of 1000
building tree 389 of 1000building tree 390 of 1000

building tree 391 of 1000
building tree 392 of 1000
building tree 393 of 1000
building tree 394 of 1000
building tree 395 of 1000building tree 396 of 1000

building tree 397 of 1000
building tree 398 of 1000
building tree 399 of 1000
building tree 400 of 1000building tree

[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:    9.4s


building tree 652 of 1000
building tree 653 of 1000building tree 654 of 1000

building tree 655 of 1000
building tree 656 of 1000building tree 657 of 1000building tree 658 of 1000


building tree 659 of 1000
building tree 660 of 1000
building tree 661 of 1000
building tree 662 of 1000building tree 663 of 1000

building tree 664 of 1000building tree 665 of 1000

building tree 666 of 1000
building tree 667 of 1000
building tree 668 of 1000
building tree 669 of 1000
building tree 670 of 1000
building tree 671 of 1000
building tree 672 of 1000building tree 673 of 1000
building tree 674 of 1000

building tree 675 of 1000
building tree 676 of 1000
building tree 677 of 1000
building tree 678 of 1000building tree 679 of 1000

building tree 680 of 1000
building tree 681 of 1000
building tree 682 of 1000
building tree 683 of 1000
building tree 684 of 1000
building tree 685 of 1000
building tree 686 of 1000
building tree 687 of 1000
building tree 688 of 1000
building tree 689 of 1000
building tre

[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   14.8s finished


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=1234, shuffle=True),
                   estimator=RandomForestClassifier(n_jobs=-1,
                                                    random_state=1234,
                                                    verbose=2),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [2, 5, 10, 20, 100],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [10, 100, 500, 1000]},
                   random_state=1234, scoring='accuracy', verbose=3)

In [59]:
rf_onehot.best_score_
### 0.6577930045976256

0.6577930045976256

In [60]:
rf_onehot.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=100, max_features='log2',
                       min_samples_leaf=2, min_samples_split=10,
                       n_estimators=1000, n_jobs=-1, random_state=1234,
                       verbose=2)

In [61]:
rf_parameter=[]
a_tmp=list(x_train.columns)
b_tmp=list(rf_onehot.best_estimator_.feature_importances_)
for i in range(len(a_tmp)):
    rf_parameter.append([a_tmp[i],b_tmp[i]])
rf_parameter=pd.DataFrame(rf_parameter)
rf_parameter=rf_parameter.sort_values(by=[1],ascending=False)

In [65]:
for i in range(len(rf_parameter)):
    print(rf_parameter.iloc[i,].values)

['All_time_peak' 0.12997536335586107]
['days_after_release' 0.09341584092782974]
['recommendations' 0.08276625274804893]
['achievements' 0.07627701443191016]
['price' 0.06680009711847412]
['initialprice' 0.06517038990003052]
['num_lang' 0.059517117577266326]
['24_Hour_Peak' 0.04406976553758533]
['average_forever' 0.04231961239447913]
['median_forever' 0.04181712168073908]
['Genre_class1' 0.03566435293894283]
['dlc' 0.028117647620555266]
['Simulation' 0.01771520009413671]
['Casual' 0.015096507552246823]
['is_free' 0.014881327782015962]
['Adventure' 0.014506239343041388]
['Action' 0.014483009030059862]
['platforms_num' 0.014141738227387166]
['controller_support' 0.013639787140246898]
['Indie' 0.013367278545477439]
['FreetoPlay' 0.013352166074397261]
['Genre_class2' 0.013330011773528234]
['Strategy' 0.012689871042722874]
['RPG' 0.011762710824787912]
['mac' 0.009498043151856582]
['EarlyAccess' 0.009488356285403687]
['MassivelyMultiplayer' 0.00686330725293869]
['linux' 0.006824051375264654]